# MongoDB with pymongo -- primer

The following introduction in several basic commands are highly influenced by the guide found on the official documentary webpage https://docs.mongodb.com/getting-started/python/introduction/

After loading the module we connect to a client. Here we use the default local one, localhost:27017. Of course it could be a remote server as well.

In [1]:
from pymongo import MongoClient
import urllib, json
client = MongoClient('ds257245.mlab.com',57245)

print(client)

MongoClient(host=['ds257245.mlab.com:57245'], document_class=dict, tz_aware=False, connect=True)


Next we use pymongo to assign a database (groups of collections) called 'tutorial'.

In [2]:
#db = client.test # attribute style
db = client ['']
db.authenticate('', '')
print(db)

Database(MongoClient(host=['ds257245.mlab.com:57245'], document_class=dict, tz_aware=False, connect=True), 'test3')


Next we construct a collection (which corresponds to tables in relational databases, groups of documents) in the database.

In [4]:
#coll = db.dataset # attribute style
coll = db['test3'] # dictionary style
print(coll)

Collection(Database(MongoClient(host=['ds257245.mlab.com:57245'], document_class=dict, tz_aware=False, connect=True), 'test3'), 'test3')


Remark: if the database doesn't exist already, MongoDB creates it automatically.

# Queries

Now we have the collection with specifier restaurants in our database. We can send some queries using pymongo and the method find(). It returns results in a cursor. find() without arguments returns all documents in the collection. We can specify some argument if we are interested in just particular documents/restaurants. Let's say we want to have all steak houses in NYC

In [5]:


cursor = db.restaurants.find({'cuisine': 'Steak'})
for documents in cursor:
    print(documents)

{'cuisine': 'Steak', '_id': ObjectId('5a04e8af81318b19b44f0e03')}
{'cuisine': 'Steak', '_id': ObjectId('5a04e8b981318b19b44f0e04')}
{'borough': 'Brooklyn', 'address': {'building': '178', 'street': 'Broadway', 'coord': [-73.96252129999999, 40.7098035], 'zipcode': '11211'}, 'grades': [{'grade': 'A', 'date': datetime.datetime(2014, 3, 8, 0, 0), 'score': 12}, {'grade': 'A', 'date': datetime.datetime(2013, 9, 28, 0, 0), 'score': 10}, {'grade': 'A', 'date': datetime.datetime(2013, 3, 26, 0, 0), 'score': 3}, {'grade': 'A', 'date': datetime.datetime(2012, 9, 10, 0, 0), 'score': 12}, {'grade': 'A', 'date': datetime.datetime(2011, 8, 15, 0, 0), 'score': 13}], '_id': ObjectId('5a04ecc32a4a84fb56330d10'), 'cuisine': 'Steak', 'name': 'Peter Luger Steakhouse', 'restaurant_id': '40364335'}
{'borough': 'Manhattan', 'address': {'building': '845', 'street': 'Lexington Avenue', 'coord': [-73.965531, 40.765431], 'zipcode': '10065'}, 'grades': [{'grade': 'A', 'date': datetime.datetime(2014, 3, 26, 0, 0), '

That is probably a little bit two much though. So let's search for steak houses in the Lexington Avenue only (logically AND). Note the dot notation for the embedded property street.

In [ ]:
cursor = db.restaurants.find({'cuisine': 'Steak', 'address.street': 'Lexington Avenue'})
for documents in cursor:
    print(documents)

We can use a comparison operator in queries as well, here we want one pizza place with a score > 30

In [ ]:
cursor = db.restaurants.find_one({'grades.score': {'$gt': 30}, 'cuisine': 'Pizza'})
print(cursor)

Exercise: search for crappy (with a score less than 5) italian or french restaurants in Queens.

In [ ]:
cursor = db.restaurants.find({'$and':[{'$or': [{'cuisine': 'Italian'}, {'cuisine': 'French'}]} \
                                      ,{'borough': 'Queens'}, {'grades.score': {'$lt': 5}}]})
for documents in cursor:
    print(documents)

We can sort the output using the following pymongo routines

In [32]:
import pymongo
cursor = db.restaurants.find({'borough': 'Manhattan', 'cuisine': 'Irish'}).sort([
    ('address.zipcode', pymongo.ASCENDING),
    ('address.coord[0]', pymongo.DESCENDING)
])
for document in cursor:
    print(document)

{u'cuisine': u'Irish', u'borough': u'Manhattan', u'name': u'Blarney Rock', u'restaurant_id': u'40366379', u'grades': [{u'date': datetime.datetime(2014, 8, 15, 0, 0), u'grade': u'A', u'score': 12}, {u'date': datetime.datetime(2014, 1, 21, 0, 0), u'grade': u'A', u'score': 5}, {u'date': datetime.datetime(2013, 7, 24, 0, 0), u'grade': u'A', u'score': 9}, {u'date': datetime.datetime(2012, 5, 31, 0, 0), u'grade': u'A', u'score': 8}, {u'date': datetime.datetime(2012, 1, 26, 0, 0), u'grade': u'A', u'score': 13}, {u'date': datetime.datetime(2011, 10, 11, 0, 0), u'grade': u'A', u'score': 5}], u'address': {u'building': u'137', u'street': u'West   33 Street', u'zipcode': u'10001', u'coord': [-73.98926, 40.7509054]}, u'_id': ObjectId('5a04ecc32a4a84fb56330d68')}
{u'cuisine': u'Irish', u'borough': u'Manhattan', u'name': u'Twins Pub', u'restaurant_id': u'40367179', u'grades': [{u'date': datetime.datetime(2014, 7, 1, 0, 0), u'grade': u'A', u'score': 10}, {u'date': datetime.datetime(2014, 1, 28, 0, 0),

Some exercise query and sort

# Manipulations of documents in the collection

Insert a new document with an artificial restaurant.

In [33]:
cursor = db.restaurants.find({'name': 'Space Cookie'})
for documents in cursor:
    print(documents)

from datetime import datetime
new_restaurant = {
        "address": {
            "street": "Broadway",
            "zipcode": "07087",
            "building": "4711",
            "coord": [-74.017212, 40.779185]
        },
        "borough": "Manhattan",
        "cuisine": "Cypermeals",
        "grades": [
            {
                "date": datetime.strptime("2017-11-10", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            }
        ],
        "name": "Space Cookie",
        "restaurant_id": "87654321"
    }
new_one = db.restaurants.insert_one(new_restaurant)
print(new_one)
print(new_one.inserted_id)
cursor = db.restaurants.find({'name': 'Space Cookie'})
for documents in cursor:
    print(documents)

5a04ed1881318b19b44f0e06
{u'cuisine': u'Cypermeals', u'borough': u'Manhattan', u'name': u'Space Cookie', u'restaurant_id': u'87654321', u'grades': [{u'date': datetime.datetime(2017, 11, 10, 0, 0), u'grade': u'A', u'score': 11}], u'address': {u'building': u'4711', u'street': u'Broadway', u'zipcode': u'07087', u'coord': [-74.017212, 40.779185]}, u'_id': ObjectId('5a04ed1881318b19b44f0e06')}


Update a document, here: change Panchito's meal style from Mexican to Spanish

In [13]:
cursor = db.restaurants.find({'name': 'Panchito\'S'})
for documents in cursor:
    print(documents)
result = db.restaurants.update_one({'name': 'Panchito\'S'}, {'$set': {'cuisine': 'Spanish'}})
print('after the update')
cursor = db.restaurants.find({'name': 'Panchito\'S'})
for documents in cursor:
    print(documents)

{'_id': ObjectId('59f9db73212f48e386d88272'), 'address': {'building': '103', 'coord': [-74.001043, 40.729795], 'street': 'Macdougal Street', 'zipcode': '10012'}, 'borough': 'Manhattan', 'cuisine': 'Mexican', 'grades': [{'date': datetime.datetime(2014, 5, 22, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 10, 10, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2013, 3, 20, 0, 0), 'grade': 'A', 'score': 13}, {'date': datetime.datetime(2012, 5, 17, 0, 0), 'grade': 'B', 'score': 20}], 'name': "Panchito'S", 'restaurant_id': '40365348'}
after the update
{'_id': ObjectId('59f9db73212f48e386d88272'), 'address': {'building': '103', 'coord': [-74.001043, 40.729795], 'street': 'Macdougal Street', 'zipcode': '10012'}, 'borough': 'Manhattan', 'cuisine': 'Spanish', 'grades': [{'date': datetime.datetime(2014, 5, 22, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2013, 10, 10, 0, 0), 'grade': 'A', 'score': 9}, {'date': datetime.datetime(2013, 3, 20, 0, 

We can also update multiple documents at once.

In [14]:
result = db.restaurants.update_many(
    {"address.zipcode": "10016", "cuisine": "Other"},
    {
        "$set": {"cuisine": "Category To Be Determined"},
        "$currentDate": {"lastModified": True}
    }
)
print(result.matched_count)
print(result.modified_count)

20
20


Note: MongoDB has a build-in functionality to update if the document exists or to create a new one with the flag {upsert:True}.

Remove a document (here we do the process in single steps, could also use find_one_and_delete to combine the action).

In [15]:
cursor = db.restaurants.find({'name': 'Windjammers Bar'})
for documents in cursor:
    print(documents)

{'_id': ObjectId('59f9db73212f48e386d883d3'), 'address': {'building': '552', 'coord': [-73.9070012, 40.7090773], 'street': 'Grandview Avenue', 'zipcode': '11385'}, 'borough': 'Queens', 'cuisine': 'American', 'grades': [{'date': datetime.datetime(2014, 9, 9, 0, 0), 'grade': 'B', 'score': 14}, {'date': datetime.datetime(2014, 1, 7, 0, 0), 'grade': 'B', 'score': 18}, {'date': datetime.datetime(2012, 11, 29, 0, 0), 'grade': 'A', 'score': 10}, {'date': datetime.datetime(2011, 12, 14, 0, 0), 'grade': 'A', 'score': 10}], 'name': 'Windjammers Bar', 'restaurant_id': '40377908'}


In [16]:
db.restaurants.delete_one({'name': 'Windjammers Bar'})
cursor = db.restaurants.find_one({'name': 'Windjammers Bar'})
print(cursor)

None


# Data aggregation

If we are interested in the number of restaurants in the single borough we can aggregate them.

In [34]:
cursor = db.restaurants.aggregate([
    {'$group': {'_id': '$borough', 'count': {'$sum': 1}}}])
for document in cursor:
    print(document)

{u'count': 51, u'_id': u'Missing'}
{u'count': 969, u'_id': u'Staten Island'}
{u'count': 2, u'_id': None}
{u'count': 10260, u'_id': u'Manhattan'}
{u'count': 6086, u'_id': u'Brooklyn'}
{u'count': 2338, u'_id': u'Bronx'}
{u'count': 5656, u'_id': u'Queens'}


exersise: group by cuisine

In [35]:
cursor = db.restaurants.aggregate([
    {'$group': {'_id': '$cuisine', 'count': {'$sum': 1}}}])
for document in cursor:
    print(document)

{u'count': 1, u'_id': u'Chilean'}
{u'count': 3, u'_id': u'Hawaiian'}
{u'count': 9, u'_id': u'Southwestern'}
{u'count': 16, u'_id': u'Hotdogs/Pretzels'}
{u'count': 16, u'_id': u'Australian'}
{u'count': 45, u'_id': u'Salads'}
{u'count': 6, u'_id': u'Czech'}
{u'count': 36, u'_id': u'Bangladeshi'}
{u'count': 31, u'_id': u'Pakistani'}
{u'count': 68, u'_id': u'Peruvian'}
{u'count': 51, u'_id': u'Soups & Sandwiches'}
{u'count': 6, u'_id': u'Nuts/Confectionary'}
{u'count': 2, u'_id': u'Caf\xc3\xa9/Coffee/Tea'}
{u'count': 66, u'_id': u'Vietnamese/Cambodian/Malaysia'}
{u'count': 14, u'_id': u'Afghan'}
{u'count': 26, u'_id': u'Brazilian'}
{u'count': 273, u'_id': u'Juice, Smoothies, Fruit Salads'}
{u'count': 26, u'_id': u'Filipino'}
{u'count': 28, u'_id': u'Tapas'}
{u'count': 59, u'_id': u'Chinese/Japanese'}
{u'count': 7, u'_id': u'Fruits/Vegetables'}
{u'count': 24, u'_id': u'Creole'}
{u'count': 8, u'_id': u'Indonesian'}
{u'count': 8, u'_id': u'Portuguese'}
{u'count': 459, u'_id': u'Sandwiches'}
{

Delete documents, here all the entries that are listed with 'Missing' for the borough key.

In [22]:
result = db.restaurants.delete_many({"borough": "Missing"})
result.deleted_count

0